In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from FRVRS import fu
from FRVRS import nu
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

In [3]:

nu.open_path_in_notepad('~/anaconda3/lib/python3.11/calendar.py', verbose=True)

Attempting to open /home/dbabbitt/anaconda3/lib/python3.11/calendar.py


In [5]:

# Print out a section of the .gitignore
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git']
print()
print('# exclude everything except personal directories')
print('/*')
print('!.gitignore')
for sub_directory in os.listdir(nu.github_folder):
    if all(map(lambda x: x not in sub_directory, black_list)) and osp.isdir(osp.join(nu.github_folder, sub_directory)): print(f'!/{sub_directory}')
print('!README.md')


# exclude everything except personal directories
/*
!.gitignore
!/builders
!/data
!/explorations
!/ipynb
!/maintenance
!/metrics
!/ps1
!/py
!/saves
!/sh
!/visualizations
!README.md


In [5]:

print([f'os.{fn}' for fn in dir(os) if 'dir' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'dir' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'github' in fn])

['os.DirEntry', 'os.O_DIRECT', 'os.O_DIRECTORY', 'os.ST_NODIRATIME', 'os.chdir', 'os.curdir', 'os.fchdir', 'os.listdir', 'os.makedirs', 'os.mkdir', 'os.pardir', 'os.removedirs', 'os.rmdir', 'os.scandir', 'os.supports_dir_fd']
['osp.curdir', 'osp.dirname', 'osp.isdir', 'osp.pardir']
['nu.github_folder']


In [6]:

for root_dir in [osp.dirname(nu.github_folder), r'C:\Users\DaveBabbitt\anaconda3\envs']:
    for sub_directory, directories_list, files_list in os.walk(root_dir):
        if ('spacy' in directories_list):
            !start %windir%\explorer.exe "{sub_directory}"
        if all(map(lambda x: x not in sub_directory, black_list)):
            for file_name in files_list:
                if ('en_core_web_sm' in file_name):
                    file_path = osp.join(sub_directory, file_name)
                    print(file_path)

In [ ]:

columns_set = set()
for pickle_name in [fn.split('.')[0] for fn in os.listdir(s.saves_pickle_folder)]:
    if pickle_name.endswith('_df'):
        df = nu.load_object(pickle_name)
        columns_set.update(df.columns[list(map(lambda x: ('year' in x) and (('begin' in x) or ('end' in x)), df.columns))])
columns_set

In [7]:

# 'Intel64 Family 6 Model 60 Stepping 3, GenuineIntel'
import platform

platform.processor()

'AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD'